# Lab | Revisiting Machine Learning Case Study

In this lab, you will use learningSet.csv file which you already have cloned in today's activities.

### 1. Check for null values in all the columns

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)



In [2]:
data = pd.read_csv('learningSet.csv')
data

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,,,0,0,,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,DCD,1,TX,77379,,,5001,0,,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,MBC,1,MI,48910,,,3801,0,,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,PRV,0,CA,91320,,,4005,0,X,...,18.0,1,L,4,F,X,X,X,11.0,A


In [3]:
# check for all nulls in df

nulls = pd.DataFrame(data.isna().sum()*100/len(data), columns=['percentage'])
nulls = nulls.sort_values('percentage', ascending = False).head(500)
nulls

,percentage
RDATE_5,99.990567
RAMNT_5,99.990567
RDATE_3,99.746363
RAMNT_3,99.746363
RDATE_4,99.705488
...,...
ETHC3,0.000000
ETHC2,0.000000
ETHC1,0.000000
HHD12,0.000000


In [4]:
# here i just made a new df for all the columns which have over 50% nulls so it's easier to analyse instead of having 480 rows we just have 60
lst = []

for i,row in nulls.iterrows():
    if row['percentage'] > 50:
        lst.append(row)
        nulls_over50 = pd.DataFrame(lst)
        



In [5]:
nulls_over50[:100]

,percentage
RDATE_5,99.990567
RAMNT_5,99.990567
RDATE_3,99.746363
RAMNT_3,99.746363
RDATE_4,99.705488
RAMNT_4,99.705488
RAMNT_6,99.186685
RDATE_6,99.186685
RAMNT_15,92.388798
RDATE_15,92.388798


### 2. Exclude the following variables by looking at the definitions. Create a new empty list called drop_list. We will append this list and then drop all the columns in this list later:

- `OSOURCE` - symbol definitions not provided, too many categories
- `ZIP CODE` - we are including state already


In [6]:
drop_list = ['OSOURCE', 'ZIP']

### 3. Identify columns that over 85% missing values

In [7]:
# i will just use the same loop from above but change the threshold to 85

lst = []

for i,row in nulls.iterrows():
    if row['percentage'] > 85:
        lst.append(row)
        nulls_over85 = pd.DataFrame(lst)

nulls_over85

,percentage
RDATE_5,99.990567
RAMNT_5,99.990567
RDATE_3,99.746363
RAMNT_3,99.746363
RDATE_4,99.705488
RAMNT_4,99.705488
RAMNT_6,99.186685
RDATE_6,99.186685
RAMNT_15,92.388798
RDATE_15,92.388798


### 4. Remove those columns from the dataframe

In [8]:
# have to swap the rows and columns so i can make a list of the columsn to drop

to_delete = list(nulls_over85.transpose())
to_delete


['RDATE_5',
 'RAMNT_5',
 'RDATE_3',
 'RAMNT_3',
 'RDATE_4',
 'RAMNT_4',
 'RAMNT_6',
 'RDATE_6',
 'RAMNT_15',
 'RDATE_15',
 'RDATE_23',
 'RAMNT_23',
 'RDATE_20',
 'RAMNT_20',
 'RAMNT_7',
 'RDATE_7',
 'RAMNT_17',
 'RDATE_17',
 'RDATE_21',
 'RAMNT_21',
 'RAMNT_10',
 'RDATE_10',
 'RDATE_13',
 'RAMNT_13',
 'NUMCHLD']

In [19]:
# drop cols and make new copy of df

data_copy = data.drop(columns= [col for col in to_delete])
data_copy

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,,,0,0,,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,DCD,1,TX,77379,,,5001,0,,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,MBC,1,MI,48910,,,3801,0,,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,PRV,0,CA,91320,,,4005,0,X,...,18.0,1,L,4,F,X,X,X,11.0,A


### 5. Reduce the number of categories in the column GENDER. The column should only have either "M" for males, "F" for females, and "other" for all the rest

Note that there are a few null values in the column. We will first replace those null values using the code below:

```python
print(categorical['GENDER'].value_counts())
categorical['GENDER'] = categorical['GENDER'].fillna('F')
```

In [20]:
data_copy['GENDER'] = data_copy['GENDER'].fillna('F')

In [21]:
data_copy['GENDER'].value_counts()

F    51277
M    39094
      2957
U     1715
J      365
A        2
C        2
Name: GENDER, dtype: int64

In [22]:
# some rows are blanks not nulls so need can't use the fillna function

data_copy['GENDER'].isna().sum()

0

In [23]:
replace = {
    'U': "other",
    'J' : "other",
    'A' : "other",
    'C' : "other",
    ' ' : "other"
}

data_copy["GENDER"] = data_copy["GENDER"].replace(replace)



i tried to do the above with a for loop but still don't really understand how to loop through dataframes :(
in this case using a dictionary is ok because there are only 5 values to replace but if there were more a loop would be more efficient.

In [24]:
# now other has replaced all the non M or F rows

data_copy['GENDER'].value_counts()

F        51277
M        39094
other     5041
Name: GENDER, dtype: int64